In [1]:
"""
We use following lines because we are running on Google Colab
If you are running notebook on a local computer, you don't need this cell
"""
from google.colab import drive
drive.mount('/content/gdrive')
import os
os.chdir('/content/gdrive/My Drive/finch/tensorflow1/free_chat/chinese/main')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
!pip install -q requests

import numpy as np
import requests
import json
import os

In [3]:
!echo "deb http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal" | tee /etc/apt/sources.list.d/tensorflow-serving.list && \
curl https://storage.googleapis.com/tensorflow-serving-apt/tensorflow-serving.release.pub.gpg | apt-key add -
!apt update
!apt-get install tensorflow-model-server

deb http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2943  100  2943    0     0   143k      0 --:--:-- --:--:-- --:--:--  143k
OK
Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease
Hit:2 http://storage.googleapis.com/tensorflow-serving-apt stable InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:8 http://security.

In [0]:
os.environ["MODEL_DIR"] = '/content/gdrive/My Drive/finch/tensorflow1/free_chat/chinese/model/gru_seq2seq_export'

In [5]:
%%bash --bg 
nohup tensorflow_model_server \
  --rest_api_port=8508 \
  --model_name=fashion_model \
  --model_base_path="${MODEL_DIR}" >server.log 2>&1

Starting job # 0 in a separate thread.


In [6]:
!tail server.log

2019-10-08 07:07:48.360162: I external/org_tensorflow/tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2019-10-08 07:07:48.400487: I external/org_tensorflow/tensorflow/cc/saved_model/loader.cc:202] Restoring SavedModel bundle.
2019-10-08 07:07:48.501425: I external/org_tensorflow/tensorflow/cc/saved_model/loader.cc:151] Running initialization op on SavedModel bundle at path: /content/gdrive/My Drive/finch/tensorflow1/free_chat/chinese/model/gru_seq2seq_export/1569460583
2019-10-08 07:07:48.524402: I external/org_tensorflow/tensorflow/cc/saved_model/loader.cc:311] SavedModel load for tags { serve }; Status: success. Took 179840 microseconds.
2019-10-08 07:07:48.525142: I tensorflow_serving/servables/tensorflow/saved_model_warmup.cc:103] No warmup data file found at /content/gdrive/My Drive/finch/tensorflow1/free_chat/chinese/model/gru_seq2seq_export/1569460583/assets.extra/tf_serving_warmup_re

In [0]:
def get_vocab(f_path):
  k2v = {}
  with open(f_path) as f:
    for i, line in enumerate(f):
      line = line.rstrip('\n')
      k2v[line] = i
  return k2v

parse_fn = lambda text: [[CHAR2IDX.get(c, len(CHAR2IDX)) for c in list(text)]]

In [8]:
CHAR2IDX = get_vocab('../vocab/char.txt')
data = json.dumps({"signature_name": "serving_default", "instances": parse_fn('你是谁')})
print(data)

{"signature_name": "serving_default", "instances": [[10, 13, 119]]}


In [9]:
headers = {"content-type": "application/json"}
json_response = requests.post('http://localhost:8508/v1/models/fashion_model:predict', data=data, headers=headers)
print(json_response)
predictions = json.loads(json_response.text)['predictions']

<Response [200]>


In [10]:
predictions = np.asarray(predictions)
IDX2CHAR = {idx: char for char, idx in CHAR2IDX.items()}
for j in range(5):
  print('A{}:'.format(j+1), ' '.join([IDX2CHAR.get(idx, len(IDX2CHAR)) for idx in predictions[0, :, j]]).replace('<end>', ''))

A1: 我 是 小 通                  
A2: 我 是 小 黄 鹰                 
A3: 不 是 她 们 俩 吗 ？               
A4: 我 是 小 通 ~                 
A5: 我 是 宇 宙 无 敌 可 爱 小 通            
